In [3]:
import pandas as pd
from dash import dash, dash_table, Input, Output, dcc, html, State, callback_context
import dash_bootstrap_components as dbc
import plotly.express as px
import numpy as np
from dash.exceptions import PreventUpdate
import time

# Cargar y preparar los datos
metadata = pd.read_csv(r"C:\Users\danie\Music\metadata.csv", skiprows=1, header=None)
titulos = ['SOP Instance UID', 'Series Instance UID', 'SOP Instance UID.1',
           'Patients Age', 'View Position', 'Image Laterality',
           'Photometric Interpretation', 'Rows', 'Columns', 'Imager Pixel Spacing',
           'Pixel Spacing', 'Pixel Padding Value', 'Pixel Padding Range Limit',
           'Window Center', 'Window Width', 'Rescale Intercept', 'Rescale Slope',
           'Rescale Type', 'Window Center & Width Explanation', 'Manufacturer',
           'Manufacturers Model Name']
metadata.columns = titulos
metadata = metadata[['SOP Instance UID', 'Series Instance UID', 'Patients Age']]
fin_anota = pd.read_csv(r"C:\Users\danie\Music\finding_annotations.csv")
metadata = metadata.drop_duplicates('Series Instance UID')
fin_anota = fin_anota.merge(metadata[['Series Instance UID', 'Patients Age']], how='left', left_on='series_id', right_on='Series Instance UID')
fin_anota['Patients Age'] = fin_anota['Patients Age'].str[:-1].fillna(0).astype(int)
fin_anota['finding_categories'] = fin_anota['finding_categories'].str[:-2].str[2:]

# DataFrame inicial para insertar nuevos registros
columns = ['study_id', 'breast_density', 'finding_categories', 'finding_birads', 'Patients Age', 'view_position', 'laterality']
df = pd.DataFrame(columns=columns)

# Crear el gráfico de barras
tabla1 = fin_anota[fin_anota['Patients Age'] > 0]
condiciones = [
    tabla1['Patients Age'].between(14, 25),
    tabla1['Patients Age'].between(25, 36),
    tabla1['Patients Age'].between(36, 47),
    tabla1['Patients Age'].between(47, 58),
    tabla1['Patients Age'].between(58, 69),
    tabla1['Patients Age'].between(69, 80),
    tabla1['Patients Age'].between(80, 91),
]
valores = ["14-25", "25-36", "36-47", "47-58", "58-69", "69-80", "80-91"]
tabla1["Edades"] = np.select(condiciones, valores)
Grafico1_count = tabla1['Edades'].value_counts().reset_index()
Grafico1_count.columns = ['Edades', 'count']
fig1 = px.bar(data_frame=Grafico1_count, x='Edades', y='count', title='Cantidad vs Edad', color='Edades')
fig1.update_layout(title={'text': 'Cantidad vs Edad', 'x': 0.5, 'xanchor': 'center'})

# Otros gráficos
tabla2 = fin_anota[fin_anota['Patients Age'] > 0]
Grafico2_count = tabla2['breast_birads'].value_counts().reset_index()
Grafico2_count.columns = ['breast_birads', 'count']
fig2 = px.pie(data_frame=Grafico2_count, values='count', names='breast_birads', hover_data={'count': True}, hole=0.3)
fig2.update_layout(title={'text': '% por BI-RADS', 'x': 0.5, 'xanchor': 'center'})
fig3 = px.pie(data_frame=fin_anota[fin_anota['Patients Age'] > 0], names='breast_density', values=None).update_layout(title={'text': 'Participación por Densidad', 'x': 0.5, 'xanchor': 'center'})
fin_anota['finding_categories'] = fin_anota['finding_categories'].str.split("'", expand=True)[0].str.capitalize()
tabla4 = fin_anota[fin_anota['Patients Age'] > 0].groupby(['finding_categories'])['finding_categories'].count().reset_index(name='conteo')
tabla4 = tabla4.sort_values(by='conteo', ascending=False)
fig4 = px.bar(data_frame=tabla4, x='conteo', y='finding_categories', orientation='h', text='conteo', color='finding_categories').update_layout(title={'text': 'Hallazgos', 'x': 0.5, 'xanchor': 'center'}).update_xaxes(title_text='Cantidad').update_yaxes(title_text='Tipo Hallazgo')
tabla5 = fin_anota[fin_anota['Patients Age'] > 0][['breast_birads', 'breast_density']]
tabla5 = tabla5.groupby(['breast_birads', 'breast_density'])['breast_density'].count().reset_index(name='Cantidad')
fig5 = px.bar(data_frame=tabla5, facet_col='breast_birads', x='breast_density', y='Cantidad', color='breast_density').update_layout(title={'text': 'Densidad por BI_RADS', 'x': 0.5}).update_xaxes(title={'text': ''})
tabla6 = fin_anota[fin_anota['Patients Age'] > 0]
condiciones = [
    tabla6['Patients Age'].between(14, 25),
    tabla6['Patients Age'].between(25, 36),
    tabla6['Patients Age'].between(36, 47),
    tabla6['Patients Age'].between(47, 58),
    tabla6['Patients Age'].between(58, 69),
    tabla6['Patients Age'].between(69, 80),
    tabla6['Patients Age'].between(80, 91),
]
valores = ["14-25", "25-36", "36-47", "47-58", "58-69", "69-80", "80-91"]
tabla6["Edades"] = np.select(condiciones, valores)
tabla6 = tabla6.groupby(['breast_birads', 'Edades'])['Edades'].count().reset_index(name='Cantiad')
fig6 = px.bar(data_frame=tabla6, facet_col='breast_birads', x='Edades', y='Cantiad', color='Edades').update_layout(title={'text': 'Rango Edad por BI_RADS', 'x': 0.5}).update_xaxes(title={'text': 'Rango_Edad'})
tabla7 = fin_anota[fin_anota['Patients Age'] > 0]
tabla7 = tabla7.groupby(['breast_birads', 'finding_categories'])['breast_birads'].count().reset_index(name='Cantidad')
fig7 = px.bar(data_frame=tabla7, facet_col='breast_birads', x='finding_categories', y='Cantidad', color='finding_categories').update_layout(title={'text': 'Hallazgos por BI_RADS', 'x': 0.5, 'xanchor': 'center'})

# Diseño del tablero
entrada1_consulta = dbc.Input(id='entrada1_consulta', placeholder='Ingrese un ID', style={'margin-bottom': '10px'})
buscar = dbc.Button(id='btn', children='Buscar', n_clicks=0, color='success', style={'backgroundColor': 'pink', 'color': 'purple', 'border-color': 'purple'})

entrada1_insercion = dbc.Input(id='entrada1_insercion', placeholder='Ingrese un ID para insertar', style={'margin-bottom': '10px'})
breast_density_dropdown = dcc.Dropdown(
    id='breast_density',
    options=[{'label': i, 'value': i} for i in fin_anota['breast_density'].dropna().unique()],
    placeholder="Seleccione la densidad mamaria",
    style={'margin-bottom': '10px'}
)
finding_categories_dropdown = dcc.Dropdown(
    id='finding_categories',
    options=[{'label': i, 'value': i} for i in fin_anota['finding_categories'].dropna().unique()],
    placeholder="Seleccione la categoría de hallazgo",
    style={'margin-bottom': '10px'}
)
finding_birads_dropdown = dcc.Dropdown(
    id='finding_birads',
    options=[{'label': i, 'value': i} for i in fin_anota['finding_birads'].dropna().unique()],
    placeholder="Seleccione BIRADS",
    style={'margin-bottom': '10px'}
)
age_dropdown = dcc.Dropdown(
    id='age',
    options=[{'label': i, 'value': i} for i in range(14, 92)],
    placeholder="Seleccione la edad del paciente",
    style={'margin-bottom': '10px'}
)
view_position_dropdown = dcc.Dropdown(
    id='view_position',
    options=[{'label': i, 'value': i} for i in fin_anota['view_position'].dropna().unique()],
    placeholder="Seleccione la posición de la vista",
    style={'margin-bottom': '10px'}
)
laterality_dropdown = dcc.Dropdown(
    id='laterality',
    options=[{'label': i, 'value': i} for i in fin_anota['laterality'].dropna().unique()],
    placeholder="Seleccione la lateralidad",
    style={'margin-bottom': '10px'}
)
submit_button = dbc.Button('Agregar Registro', id='submit-val', n_clicks=0, style={'margin-bottom': '10px','backgroundColor':'pink','color':'purple'})

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div(
    [
        html.Div(html.H1("ESTUDIO EPIDEMIOLÓGICO", style={'text-align': 'center', 'backgroundColor': 'pink', 'color': 'purple', 'margin-back': '10px'}), style={'backgroundColor': 'pink'}),
        dbc.Col(html.H2("Modulo de consulta", style={'color': 'purple'}), style={'margin-left': '4%', 'margin-top': '4%'}),
        dbc.Row([
            dbc.Col(entrada1_consulta, width=4, style={'margin-left': '4%', 'margin-top': '10px'}),
            dbc.Col(buscar, width=4, style={'margin-left': '2%', 'margin-top': '10px'})
        ]),
        html.Div(
            dcc.Loading(
                id="loading-spinner-consulta",
                children=html.Div(id='salida_consulta'),
                type="default",
            ),
            style={'margin-top': '4%'}
        ),
        dbc.Col(html.H2("Modulo de inserción", style={'color': 'purple'}), style={'margin-left': '4%', 'margin-top': '4%'}),
        dbc.Row([
            dbc.Col(entrada1_insercion, width=4, className='mb-3'),
            dbc.Col(breast_density_dropdown, width=4, className='mb-3'),
            dbc.Col(finding_categories_dropdown, width=4, className='mb-3'),
            dbc.Col(finding_birads_dropdown, width=4, className='mb-3'),
            dbc.Col(age_dropdown, width=4, className='mb-3'),
            dbc.Col(view_position_dropdown, width=4, className='mb-3'),
            dbc.Col(laterality_dropdown, width=4, className='mb-3'),
            dbc.Col(submit_button, width=4, className='mb-3')
        ], justify='center'),
        dbc.Row(
            dbc.Col(
                dcc.Loading(
                    id="loading-spinner-insercion",
                    children=html.Div(id='salida_insercion'),
                    type="default",
                ),
                width=10, className='my-4'
            )
        ),
        dcc.Graph(figure=fig1),
        dbc.Row([
            dbc.Col(dcc.Graph(figure=fig2), width=4, style={'margin-top': '4%'}),
            dbc.Col(dcc.Graph(figure=fig3), width=4, style={'margin-top': '4%'})
        ], style={'justify-content': 'center', 'display': 'flex'}),
        dbc.Row(dcc.Graph(figure=fig4)),
        dcc.Graph(figure=fig5),
        dcc.Graph(figure=fig6),
        dcc.Graph(figure=fig7)
    ], style={'margin-left': '5%', 'margin-right': '5%'}
)

px.box(data_frame=fin_anota, y='Patients Age')

def buscarid(id):
    filtro = fin_anota[fin_anota['study_id'].astype(str) == id][['study_id', 'breast_density', 'finding_categories', 'finding_birads', 'Patients Age', 'view_position', 'laterality']]
    return filtro

@app.callback(
    Output('salida_consulta', 'children'),
    [State('entrada1_consulta', 'value'),
     Input('btn', 'n_clicks')]
)
def buscar(entrada, click):
    if entrada is not None and click > 0:
        time.sleep(5)  # Simular la demora de 5 segundos
        filtro = fin_anota[fin_anota['study_id'].astype(str) == entrada][['study_id', 'breast_density', 'finding_categories', 'finding_birads', 'Patients Age', 'view_position', 'laterality']]
        if not filtro.empty:
            return dash_table.DataTable(filtro.to_dict('records'), [{"name": i, "id": i} for i in filtro.columns], style_header={'text-align': 'center', 'backgroundColor': 'purple', 'color': 'pink'}, style_cell={'text-align': 'center'})
        else:
            return html.Div("Registro no encontrado.", style={'color': 'red', 'font-size': '20px', 'text-align': 'center'})
    return ''

@app.callback(
    Output('salida_insercion', 'children'),
    [Input('submit-val', 'n_clicks')],
    [State('entrada1_insercion', 'value'),
     State('breast_density', 'value'),
     State('finding_categories', 'value'),
     State('finding_birads', 'value'),
     State('age', 'value'),
     State('view_position', 'value'),
     State('laterality', 'value')]
)
def update_output(n_clicks, study_id, breast_density, finding_categories, finding_birads, age, view_position, laterality):
    if n_clicks > 0:
        time.sleep(5)  # Simular la demora de 5 segundos
        new_data = {
            'study_id': study_id,
            'breast_density': breast_density,
            'finding_categories': finding_categories,
            'finding_birads': finding_birads,
            'Patients Age': age,
            'view_position': view_position,
            'laterality': laterality
        }
        global df
        df = df.append(new_data, ignore_index=True)

        # Actualizar fin_anota con el nuevo registro
        global fin_anota
        fin_anota = fin_anota.append(new_data, ignore_index=True)

        return html.Div("Registro ingresado correctamente.", style={'color': 'green', 'font-size': '20px', 'text-align': 'center'})
    return ''

if __name__ == '__main__':
    app.run_server(debug=True, port=8090)


C:\Users\danie\AppData\Local\Temp\ipykernel_9784\1570597583.py:42: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\danie\AppData\Local\Temp\ipykernel_9784\1570597583.py:73: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



C:\Users\danie\AppData\Local\Temp\ipykernel_9784\1570597583.py:219: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\danie\AppData\Local\Temp\ipykernel_9784\1570597583.py:223: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\danie\AppData\Local\Temp\ipykernel_9784\1570597583.py:219: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\danie\AppData\Local\Temp\ipykernel_9784\1570597583.py:223: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\danie\AppData\Local\Temp\ipykernel_9784\1570597583.py:219: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co